In [34]:
#from PyPDF2 import PdfReader
import PyPDF2
import pandas as pd
import openai
import os
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')
state = "Colorado"

In [43]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

psco_tar = "PSCo_Electric_Entire_Tariff.pdf"
output_string = StringIO()
with open(psco_tar, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())


ValueError: seek of closed file

In [48]:
fp = open("PSCo_Electric_Entire_Tariff.pdf", 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)

for pageNumber, page in enumerate(PDFPage.create_pages(document)):
    if pageNumber == 42:
        print(page)


<PDFPage: Resources={'ColorSpace': {'CS0': <PDFObjRef:33385>}, 'Font': {'TT0': <PDFObjRef:2152>, 'TT1': <PDFObjRef:2150>, 'TT2': <PDFObjRef:2151>, 'TT3': <PDFObjRef:2153>}}, MediaBox=[0.0, 0.0, 612.0, 792.0]>


In [45]:
page

<PDFPage: Resources={'ColorSpace': {'CS0': <PDFObjRef:33385>}, 'Font': {'TT0': <PDFObjRef:2450>, 'TT1': <PDFObjRef:2452>, 'TT2': <PDFObjRef:2451>, 'TT3': <PDFObjRef:2453>, 'TT4': <PDFObjRef:2454>}}, MediaBox=[0.0, 0.0, 612.0, 792.0]>

In [ ]:
#Original Code that was too long for pdf

psco_tar = "PSCo_Electric_Entire_Tariff.pdf"

state = 'Colorado'
# Open the PDF file in read-binary mode
with open(psco_tar, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    
    # Initialize an empty string to store the extracted text
    text = ''
    
    # Iterate through each page in the PDF and extract the text
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

# Close the PDF file
pdf_file.close()

prompt = f"""
I am a data analyst who works for an electric company.  My company has tariffs setup for pricing in ```{state}``` 
Read through the text below and return a summary Structured as below if the data is found:

"<Section>
    - <Category>: 
        <Summary by category>"

Here are the categories by section and what should be summarized in each category:

Territory
    - Counties: list of counties

Cost Structure
    - Rates: a bulleted list of "<rate name>:<cost>"
        - Seasonal - a bulleted list of seasonal changes in rate
        - Time of Use - a bulleted list of Time of Use (TOU)
        - Incentives - ie renewable, saver switch, demand response programs, etc.
    - Taxes and Fees 
    - Measurement - how the data will be measured by the meter.

Electric Market
    - Tariff Type - ie transmission, generation, distribution, surcharges
    - Demand type - ie commercial, industrial or residential

Regulatory
    - Regulatory - compliance

The text is
```{text}```
"""

def get_completion(prompt, model="gpt-4-1106-preview"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


response = get_completion(prompt)
print(response)

In [30]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBox

# Open a PDF document.
fp = open("PSCo_Electric_Entire_Tariff.pdf", 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser)
# Get the outlines of the document.
outlines = document.get_outlines()
for(level,title,dest,a,se) in outlines:
    #print (' '.join(title.split(' ')[1:]))
    print (level,title)

if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()
# Create a PDF device object.
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
text_from_pdf = open('textFromPdf.txt','w')
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    layout = device.get_result()
    for element in layout:
        if isinstance(element, LTTextBox):
            text_from_pdf.write(''.join([i if ord(i) < 128 else ' ' for i in element.get_text()]))

1 20161208132801
1 20161208133049
2 Recycled Energy
2 PURCHASED CAPACITY COST ADJUSTMENT
2 Residential Service
2 Small Commercial Service
2 Commercial & Industrial General Service
2 Special Contract Service
2 Recycled Energy
3 PURCHASED CAPACITY COST ADJUSTMENT
2 Standby Service
2 Lighting Service
2 Residential Service
2 Small Commercial Service
2 Commercial & Industrial General Service
2 Special Contract Service
2 Recycled Energy
2 Standby Service
2 Lighting Service
2 Residential Service
2 Small Commercial Service
2 Commercial & Industrial General Service
2 Special Contract Service
2 Recycled Energy
2 Standby Service
2 Lighting Service
2 ELECTRIC SALES RATES
2 Commercial and Industrial Transmission Service:
2 Transmission General  Schedule TG  70
2 Transmission General Critical Peak Pricing Schedule TG-CPP  71
2 Transmission Standby Service Schedule TST  72
2 Commercial and Industrial Special Contract:
2 Regional Transportation District Schedule SCS-7  80
2 Regional Transportation Dis

In [16]:
outlines.level()

AttributeError: 'generator' object has no attribute 'level'

In [12]:
PDFDocument("PSCo_Electric_Entire_Tariff.pdf")

NameError: name 'PDFDocument' is not defined

In [10]:
def parse(filename, maxlevel):
    fp = open(filename, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(fp)
    parser.set_document(doc)
    doc.set_parser(parser)

In [11]:
parse("PSCo_Electric_Entire_Tariff.pdf",5)

NameError: name 'PDFDocument' is not defined

In [ ]:
outlines = doc.get_outlines()
for (level, title, dest, a, se) in outlines:
    if level <= maxlevel:
        title_words = title \
                .encode('utf8') \
                .replace('\n', '') \
                .split()
        title = ' '.join(title_words)
        print(' ' * level, title)